<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Generative AI with OpenAI API</h1>
<h1>GPT Models</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from datetime import datetime
import json

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 

from openai import OpenAI
import termcolor
from termcolor import colored

import os
import gzip

import tqdm as tq
from tqdm.notebook import tqdm

import watermark

%load_ext watermark
%matplotlib inline

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.21.0

Compiler    : Clang 15.0.0 (clang-1500.1.0.2.5)
OS          : Darwin
Release     : 23.3.0
Machine     : x86_64
Processor   : i386
CPU cores   : 16
Architecture: 64bit

Git hash: 529fe8a3ea9769b549bab95c993473f8266299a8

json      : 2.0.9
termcolor : 2.4.0
tqdm      : 4.66.1
matplotlib: 3.8.2
watermark : 2.4.3
numpy     : 1.26.4
pandas    : 2.2.0



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Basic Usage

The first step is always to load up the API key from the local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key

In [5]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

We start by getting a list of supported models.

In [12]:
model_list = client.models.list().data

In total we have 25 models

In [13]:
len(model_list)

27

Along with some information about each model...

In [15]:
model_list[:3]

Model(id='dall-e-3', created=1698785189, object='model', owned_by='system')

But let's just get a list of model names

In [17]:
print("\n".join(sorted([model.id for model in model_list])))

babbage-002
dall-e-2
dall-e-3
davinci-002
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-0301
gpt-3.5-turbo-0613
gpt-3.5-turbo-1106
gpt-3.5-turbo-16k
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
gpt-4
gpt-4-0125-preview
gpt-4-0613
gpt-4-1106-preview
gpt-4-turbo-preview
gpt-4-vision-preview
text-embedding-3-large
text-embedding-3-small
text-embedding-ada-002
tts-1
tts-1-1106
tts-1-hd
tts-1-hd-1106
whisper-1


## Basic Prompt

The recommended model for exploration is `gpt-3.5-turbo`, so we'll stick with it for now. The basic setup is relatively straightforward:

In [22]:
%%time
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

CPU times: user 9.56 ms, sys: 3.72 ms, total: 13.3 ms
Wall time: 1.44 s


Which produces a response object

In [23]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

Which we can treat as a JSON object

In [25]:
pprint(response.choices)

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Superman's weakness is Kryptonite, a radioactive mineral from his home planet Krypton, which weakens and can eventually kill him.", role='assistant', function_call=None, tool_calls=None))]


The model answer can be found in the "message" dictionary inside the "choices" list

In [27]:
response.choices[0].message.content

"Superman's weakness is Kryptonite, a radioactive mineral from his home planet Krypton, which weakens and can eventually kill him."

To request multiple answers, we must include the `n` parameter with the number of answers we want

In [28]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "What are the different kinds of Kryptonite?"},
    ],
    n=3
)

And we can access each of the answers individually int he choices list

In [29]:
for output in response.choices:
    print("==========")
    print(output.message.role.title()) 
    print("==========")
    print(output.message.content)
    print("==========\n")

Assistant
In the DC Comics universe, Kryptonite is a fictional mineral that is highly dangerous to Superman and other Kryptonians, weakening or even killing them. Over the years, different variations of Kryptonite have been introduced in the comics, movies, and television series. Here are some of the notable kinds of Kryptonite:

1. Green Kryptonite: The most common and well-known form of Kryptonite, Green Kryptonite is deadly to Kryptonians. Exposure to it weakens Superman and others, sapping their powers, making them vulnerable, and potentially leading to their demise.

2. Red Kryptonite: Red Kryptonite has unpredictable effects on Kryptonians. Unlike Green Kryptonite, its effects are temporary and can vary each time a different Kryptonian is exposed to it. It often causes temporary personality changes, alter ego transformations, or temporary loss of powers.

3. Blue Kryptonite: Blue Kryptonite specifically affects Bizarro, a flawed, imperfect clone of Superman. It weakens and harms 

In [31]:
response.usage

CompletionUsage(completion_tokens=1236, prompt_tokens=17, total_tokens=1253)

# Temperature

In [36]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=1.5
)

CPU times: user 8.02 ms, sys: 2.2 ms, total: 10.2 ms
Wall time: 16.8 s


In [37]:
print(response.choices[0].message.content)

Once upon a time, in a small quiet village nestled in the heart of the forest, there lived a poor yet joyful couple named Elisa and Henry. The couple shared a little wooden cottage where they led a very modest life. Although frugal, they were content and grateful for what they had.

One day, as Elisa was walking through the dense woodland, she discovered a magic chest filled with gleaming golden coins. Astonished and filled with excitement, she hurried back to Henry, sharing the fantastic news. With glee in their eyes and hearts, they dreamt of the countless wondrous things they could afford with their newfound wealth.

After division and careful thought, Elisa and Henry made essential purchases, wisely investing the majority of the treasure while keeping just enough for their humble desires. Their cottage windows were finally adorned with colorful curtains, their table rid of tears embedded by previous days, replaced with a newfound remarkable set.

Along with material improvements, E

In [38]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a short story"},
    ],
    temperature=0
)

CPU times: user 8.36 ms, sys: 2.01 ms, total: 10.4 ms
Wall time: 10.3 s


In [39]:
print(response.choices[0].message.content)

Once upon a time, in a small village nestled in the heart of a lush forest, there lived a young girl named Lily. She was known for her kind heart and adventurous spirit. One sunny morning, as she was exploring the woods, she stumbled upon a wounded bird with a broken wing.

Lily carefully picked up the bird and cradled it in her hands. She knew she had to help it, so she decided to take it home. With great care, she fashioned a small nest for the bird and gently placed it inside. Lily named the bird Ruby, after its vibrant red feathers.

Days turned into weeks, and Lily tirelessly tended to Ruby's needs. She fed her, kept her warm, and even sang lullabies to comfort her. Slowly but surely, Ruby's wing began to heal. Lily's dedication and love had worked wonders.

One day, as Lily was preparing to release Ruby back into the wild, a sudden storm swept through the village. The wind howled, and rain poured down in torrents. Lily worried about Ruby's safety, knowing she couldn't fly yet.

W

# Function Calls

In [40]:
def chat(messages, functions):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        # Define the functions the model is allowed to use
        functions=functions
    )
    
    return response

In [55]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


Let's create some function specifications to interface with a hypothetical weather API. We'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

In [51]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    },
]

If we prompt the model about the current weather, it will respond with some clarifying questions.

In [59]:
messages = []

messages.append(
    {"role": "system", 
     "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    })

messages.append(
    {"role": "user", 
     "content": "What's the weather like today"
    })

In [60]:
chat_response = chat(messages, functions=functions)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message.__dict__)

In [61]:
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant: Sure, could you please tell me the city and state for which you would like to know the current weather?



Once we provide the missing information, it will generate the appropriate function arguments for us.

In [62]:
messages.append(
    {"role": "user", 
     "content": "I'm in New York, NY."
    })

In [64]:
chat_response = chat(messages, functions=functions)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message.__dict__)

BadRequestError: Error code: 400 - {'error': {'message': "None is not of type 'object' - 'messages.2.function_call'", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [65]:
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant: Sure, could you please tell me the city and state for which you would like to know the current weather?

user: I'm in New York, NY.



In [66]:
messages

[{'role': 'system',
  'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user', 'content': "What's the weather like today"},
 {'content': 'Sure, could you please tell me the city and state for which you would like to know the current weather?',
  'role': 'assistant',
  'function_call': None,
  'tool_calls': None},
 {'role': 'user', 'content': "I'm in New York, NY."}]

In [67]:
chat_response

ChatCompletion(id='chatcmpl-8runbyNW9rYjsM2xB4vHHNs41rldo', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Sure, could you please tell me the city and state for which you would like to know the current weather?', role='assistant', function_call=None, tool_calls=None))], created=1707860411, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=23, prompt_tokens=112, total_tokens=135))

## Few-shot prompting

We can also provide several examples of mappings between input and output.

In [31]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
        {"role": "assistant", "content": "Things working well together will increase revenue."},
        {"role": "user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response["choices"][0]["message"]["content"])

This sudden change in direction means we don't have enough time to complete the entire project for the client.


# Formatted output

In [32]:
%%time
userInput = "blueberry pancakes"

prompt = """return a recipe for %s.
        Provide your response as a JSON object with the following schema:
        {"dish": "%s", "ingredients": ["", "", ...],
        "instructions": ["", "", ... ]}""" % (userInput, userInput)

response = openai.ChatCompletion.create(
          model = "gpt-3.5-turbo",
          messages = [
            { "role": "system", "content": "You are a helpful recipe assistant."},
            { "role": "user",   "content": prompt }
          ]
)

CPU times: user 3.69 ms, sys: 1.72 ms, total: 5.42 ms
Wall time: 4.14 s


In [33]:
json_output = response["choices"][0]["message"]["content"]

In [34]:
output = json.loads(json_output)

In [35]:
output["ingredients"]

['1 cup all-purpose flour',
 '2 tablespoons granulated sugar',
 '1 1/2 teaspoons baking powder',
 '1/2 teaspoon salt',
 '3/4 cup milk',
 '1 large egg',
 '2 tablespoons unsalted butter, melted',
 '1 cup fresh blueberries',
 'Additional butter or oil for greasing the pan']

In [36]:
output["instructions"]

['In a large bowl, whisk together the flour, sugar, baking powder, and salt.',
 'In a separate bowl, whisk together the milk, egg, and melted butter.',
 'Pour the wet ingredients into the dry ingredients and stir until just combined. Do not overmix; a few lumps are okay.',
 'Gently fold in the blueberries.',
 'Heat a non-stick griddle or skillet over medium heat. Grease with butter or oil.',
 'Pour 1/4 cup of batter onto the griddle for each pancake.',
 'Cook until bubbles form on the surface of the pancake and the edges start to look set, about 2-3 minutes.',
 'Flip the pancakes and cook for an additional 1-2 minutes, or until golden brown.',
 'Repeat with the remaining batter, adding more butter or oil to the pan as needed.',
 'Serve the blueberry pancakes warm with maple syrup and additional blueberries, if desired.']

# Translation

In [37]:
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": "You're a professional English-Italian translator."}, 
              {"role": "user", "content": "Translate 'Be the change that you wish to see in the world.' into Italian"}],
    temperature=0,
)

In [38]:
response["choices"][0]["message"]["content"]

'"Sii il cambiamento che desideri vedere nel mondo."'

# Process unstructured information

Inspired by https://platform.openai.com/examples/default-parse-data

In [39]:
prompt = """There are many fruits that were found on the recently discovered planet Goocrux. 
There are neoskizzles that grow there, which are purple and taste like candy. There are also 
loheckles, which are a grayish blue fruit and are very tart, a little bit like a lemon. Pounits 
are a bright green color and are more savory than sweet. There are also plenty of loopnovas which 
are a neon pink flavor and taste like cotton candy. Finally, there are fruits called glowls, which 
have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them."""

In [40]:
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", 
         "content": "You will be provided with unstructured data, and your task is to parse it into CSV format."}, 
        {"role": "user", 
         "content": prompt}],
    temperature=0,
)

In [41]:
print(response["choices"][0]["message"]["content"])

Fruit,Color,Taste
neoskizzles,purple,candy
loheckles,grayish blue,tart
pounits,bright green,savory
loopnovas,neon pink,cotton candy
glowls,pale orange,sour and bitter


In [42]:
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": """
            Read this paragraph 
            
            `%s` 
            
            and use it to answer some questions.""" % prompt}, 
              {"role": "user", "content": "What are pounits?"}],
    temperature=0,
)

In [43]:
print(response["choices"][0]["message"]["content"])

Pounits are bright green fruits that are more savory than sweet.


<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>